# Atlas dynamique de l'expérience `keyexp`

In [ ]:
import netCDF4
from pylab import *
import numpy as np
import cartopy.crs as ccrs

ncpath = "./"
ncfile=ncpath+"resultat.nc"
data = netCDF4.Dataset(ncfile)
longi=data.variables['longitude'][:]
lati=data.variables['latitude'][:]
altitude=data.variables['altitude'][:]
Time = data.variables['Time'][:]
Ls = data.variables['Ls'][:]

dafirst = Time[0]
daint = Time[1] - dafirst
dalast = dafirst + (len(Time)-1)*daint
year = 0.
add = np.linspace(dafirst,dalast,num=len(Time)) ; add[0] = 0.
for iii in range(1,len(Ls)):
    if Ls[iii] - Ls[iii-1] < 0: year = year+1.
    add[iii] = year*360.
Ls_true = add + Ls

# Choix du $L_s$

In [ ]:
# Paramètres utilisateurs -----------------------------------------
year_user = 1 # année de simulation à regarder (démarre à 0)
Ls_user = 90. # longitude solaire choisie

# Code ------------------------------------------------------------
Ls_true_user = year_user*360. + Ls_user

Ls_ind = np.where(abs(Ls_true-Ls_true_user)==
                  abs(Ls_true-Ls_true_user).min())[0]
print("La valeur la plus proche trouvée est Ls = "
      + str(Ls_true[Ls_ind]-year_user*360.)
      + " pour l'année " + str(year_user))
# -----------------------------------------------------------------

# Choix de l'altitude

In [ ]:
# Paramètres utilisateurs -----------------------------------------
alti_user = 10 # altitude choisie en km

# Code ------------------------------------------------------------
alti_index = np.where(abs(altitude-alti_user)==
                  abs(altitude-alti_user).min())[0]
print("La valeur la plus proche trouvée est z = "
      + str(altitude[alti_index]))
# -----------------------------------------------------------------

# Vent à l'altitude choisie

In [ ]:
#------------------------------------------------------------------
# Paramètres utilisateurs

#------------------------------------------------------------------
# Code principal

ps = data.variables['ps'][:]
u = data.variables['u'][:]
v = data.variables['v'][:]

dataplt = np.squeeze(ps[Ls_ind,:,:])
uwind = np.squeeze(u[Ls_ind,alti_index,:,:])
vwind = np.squeeze(v[Ls_ind,alti_index,:,:])

fig = plt.figure(figsize=(16,12))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
#ax.coastlines(resolution="110m",linewidth=1)
gl = ax.gridlines(linestyle='--',color='black',
             draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
clevs = np.linspace(100,1300,25)
plt.contourf(longi, lati, dataplt, clevs, transform=ccrs.PlateCarree(),cmap=plt.cm.jet)
plt.title("Pression de surface et vents -  z = " 
          + str(altitude[alti_index]) + "km - Ls = "+str(Ls_true[Ls_ind]-year_user*360.), size=14)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label(r'Pa',size=12,rotation=0,labelpad=15)
cb.ax.tick_params(labelsize=10)
ax.barbs(longi, lati, uwind,vwind, length=5, pivot="middle",
         regrid_shape=20, transform=ccrs.PlateCarree(),
         barb_increments=dict(half=5, full=10, flag=20))
plt.show()

# Coupe du vent zonal

In [ ]:
# Paramètres utilisateurs -----------------------------------------
vmin = 5.
vmax = 100.
ncontour = 20

# Code ------------------------------------------------------------
vitu = data.variables['u'][:,:,:,:]
p = data.variables['p'][:,:,:,:]

fig, ax = plt.subplots(figsize=(14,6))
xplt=lati
yplt = np.squeeze(np.mean(np.mean(p[Ls_ind,:,:,:],axis=3),axis=2))
dataplt = np.squeeze(np.mean(vitu[Ls_ind,:,:,:],axis=3))
im = ax.contourf(xplt,yplt,dataplt,ncontour,cmap='jet',vmax=vmax, vmin=vmin)
#axs[0, 0].cbar()
#ax1.set_ylim([-90,90])
ax.set_title("Vent zonal - Ls = "+str(Ls_true[Ls_ind]-year_user*360.))
ax.set_yscale('log')
ax.set(xlabel='Latitude', ylabel='p')
ax.invert_yaxis()
fig.colorbar(im,ax=ax)
# -----------------------------------------------------------------

# Coupe de température

In [ ]:
# Paramètres utilisateurs -----------------------------------------
tempmin = 180.
tempmax = 360.
ncontour = 60

# Code ------------------------------------------------------------
temp = data.variables['temp'][:,:,:,:]
p = data.variables['p'][:,:,:,:]

fig, ax = plt.subplots(figsize=(14,6))
xplt=lati
yplt = np.squeeze(np.mean(np.mean(p[Ls_ind,:,:,:],axis=3),axis=2))
dataplt = np.squeeze(np.mean(temp[Ls_ind,:,:,:],axis=3))
im = ax.contourf(xplt,yplt,dataplt,ncontour,cmap='jet',vmax=tempmax, vmin=tempmin)
#axs[0, 0].cbar()
#ax1.set_ylim([-90,90])
ax.set_title("Température - Ls = "+str(Ls_true[Ls_ind]-year_user*360.))
ax.set_yscale('log')
ax.set(xlabel='Latitude', ylabel='p')
ax.invert_yaxis()
fig.colorbar(im,ax=ax)
# -----------------------------------------------------------------